In [140]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as plt
import sys
import os
sys.path.append(os.path.abspath(".."))
from utils import create_geometry_column, find_similar_phrases, normalize_street_names

ImportError: cannot import name 'normalize_street_names' from 'utils' (C:\Users\igarl\Desktop\DA_Project\utils.py)

# Data load

In [ ]:
# Loading data
df = pd.read_csv("../data/nypd-motor-vehicle-collisions.csv", low_memory=False)

# Dataset information

In [ ]:
print(df.shape)

In [ ]:
print(df.columns)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

# Data Cleaning

## Deleting unnecessary columns

In [ ]:
del df["ZIP CODE"]

## Datetime

In [ ]:
df["ACCIDENT DATE"]

In [ ]:
df["ACCIDENT DATE"] = pd.to_datetime(df["ACCIDENT DATE"], format="%Y-%m-%dT%H:%M:%S.%f")
df["ACCIDENT DATE FORMATTED"] = df["ACCIDENT DATE"].dt.strftime("%d/%m/%Y")
df["ACCIDENT DATE"] = df["ACCIDENT DATE"].dt.normalize()

In [ ]:
df["ACCIDENT DATE"].head(5)

In [ ]:
df["ACCIDENT TIME"].head(5)

## Missing boroughs, latitude, longitude and location

In [ ]:
df["BOROUGH"].unique()

In [ ]:
# Check how many boroughs are missing
missing_boroughs = df["BOROUGH"].isna().mean()
print(f"Missing boroughs {round(missing_boroughs*100,2)} %")

In [ ]:
df[["LATITUDE", "LONGITUDE"]].isna().sum()

In [ ]:
# Check if both - latitude and longitude is missing in observation
missing_count = df[df["LATITUDE"].isna() & df["LONGITUDE"].isna()].shape[0]
print(f"Missing both latitude and longitude: {missing_count}")

## Match coordinates with boroughs from geojson

### Create geometry df

In [ ]:
geometry_df = df.dropna(subset=["LONGITUDE", "LATITUDE"]).copy()
geometry_df = geometry_df[["LONGITUDE", "LATITUDE", "BOROUGH"]]
# create a column with original index as the original one will be replaced after sjoin
geometry_df["ORIGINAL INDEX"] = geometry_df.index
geometry_df.info()

### Create geometry column

In [ ]:
create_geometry_column(geometry_df)
geometry_df.info()

### Convert geometry_df into GeoDataFrame

In [ ]:
geometry_gdf = gpd.GeoDataFrame(geometry_df, geometry="GEOMETRY", crs="EPSG:4326")

### Load boroughs from geojson

In [ ]:
# Load borough boundaries
# https://www.nyc.gov/content/planning/pages/resources/datasets/borough-boundaries
boroughs_gdf = gpd.read_file("../data/nybb.shp")

In [ ]:
print(boroughs_gdf.columns)
print(boroughs_gdf.crs)

In [ ]:
boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)

### Spatial join

In [ ]:
print(boroughs_gdf.columns)

In [ ]:
geometry_gdf_joined = gpd.sjoin(geometry_gdf, boroughs_gdf[["BoroName", "geometry"]], how = "left", predicate = "intersects")

In [ ]:
print(geometry_gdf_joined["BoroName"].unique())

In [ ]:
geometry_gdf_joined["BoroName"].isna().sum()

In [ ]:
geometry_gdf_joined["BoroName"] = geometry_gdf_joined["BoroName"].fillna("Unknown")

In [ ]:
geometry_gdf_joined["BoroName"].unique()

In [ ]:
geometry_gdf_joined["BoroName"] = geometry_gdf_joined["BoroName"].astype(str).str.upper()

In [ ]:
geometry_gdf_joined.head(5)

### Fill NaN with mapped boroughs

In [ ]:
df.loc[geometry_gdf_joined["ORIGINAL INDEX"], "BOROUGH"] = df.loc[
    geometry_gdf_joined["ORIGINAL INDEX"], "BOROUGH"
    ].combine_first(geometry_gdf_joined.set_index("ORIGINAL INDEX")["BoroName"])

In [ ]:
df.info()

In [ ]:
df['BOROUGH'].unique()

## Label NaN as "Unknown"

In [ ]:
cols_to_fill = ["BOROUGH","LATITUDE","LONGITUDE","LOCATION"]
df[cols_to_fill] = df[cols_to_fill].apply(lambda col: col.fillna("UNKNOWN"))

In [ ]:
df['BOROUGH'].unique()

In [ ]:
df.info()

## Number of persons

In [ ]:
df["NUMBER OF PERSONS INJURED"].unique()

In [ ]:
df["NUMBER OF PERSONS INJURED"] = df["NUMBER OF PERSONS INJURED"].astype("Int64")

In [ ]:
df["NUMBER OF PERSONS INJURED"].unique()

In [ ]:
df["NUMBER OF PERSONS KILLED"].unique()

In [ ]:
df["NUMBER OF PERSONS KILLED"] = df["NUMBER OF PERSONS KILLED"].astype("Int64")

In [ ]:
df.info()


In [ ]:
df["NUMBER OF MOTORIST KILLED"].unique()

## String formatting

In [ ]:
cols_to_format = ["CONTRIBUTING FACTOR VEHICLE 1", "CONTRIBUTING FACTOR VEHICLE 2", "CONTRIBUTING FACTOR VEHICLE 3",
                     "CONTRIBUTING FACTOR VEHICLE 4", "CONTRIBUTING FACTOR VEHICLE 5", "VEHICLE TYPE CODE 1", 
                     "VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5",
                 "ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME"]
df[cols_to_format] = df[cols_to_format].astype(str).apply(
    lambda col: col.str.strip().str.lower())

## Streets

In [ ]:
on_street_typos = find_similar_phrases(df["ON STREET NAME"], 70)

In [ ]:
print(on_street_typos)

In [ ]:
streets_to_format = ["ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME"]
df[streets_to_format] = df[streets_to_format].applymap(normalize_street_names)

## Contributing factors

In [ ]:
cfv_typos_cols_1 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 1"])
cfv_typos_cols_2 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 2"])
cfv_typos_cols_3 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 3"])
cfv_typos_cols_4 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 4"])
cfv_typos_cols_5 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 5"])

In [ ]:
print(f"CFV 1 typos {cfv_typos_cols_1}")
print(f"CFV 2 typos {cfv_typos_cols_2}")
print(f"CFV 3 typos {cfv_typos_cols_3}")
print(f"CFV 4 typos {cfv_typos_cols_4}")
print(f"CFV 5 typos {cfv_typos_cols_5}")

### Corrections directory

In [ ]:
df["CONTRIBUTING FACTOR VEHICLE 1"].unique()

In [ ]:
corrections_cfv = {"reaction to other uninvolved vehicle":"reaction to uninvolved vehicle", "illnes":"illness", "nan":"unspecified", 
                  "80": "unspecified", "1": "unspecified"}

In [ ]:
cfv_cols_to_correct = ["CONTRIBUTING FACTOR VEHICLE 1", "CONTRIBUTING FACTOR VEHICLE 2", "CONTRIBUTING FACTOR VEHICLE 3",
                     "CONTRIBUTING FACTOR VEHICLE 4", "CONTRIBUTING FACTOR VEHICLE 5"]
df[cfv_cols_to_correct] = df[cfv_cols_to_correct].replace(corrections_cfv)

In [ ]:
cfv_typos_cols_1_cleaned = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 1"])
print(cfv_typos_cols_1_cleaned)

## Vehicle types

In [ ]:
df.info()